In [4]:
import tensorflow as tf
import numpy as np

In [7]:
tf.__version__

'1.1.0'

In [62]:
class FFCell:
    def __init__(self, input_size, output_size, hidden_desc, regression = False):
        self.input_size = input_size
        self.output_size = output_size
        self.hidden_desc = hidden_desc
        self.regression = regression
        self.layers = []
#         self._build_net()
    
    def _build_net(self):
        self.layers.append({ 'weights': tf.Variable(tf.random_normal([self.input_size, self.hidden_desc[0]['hidden_nodes']])),
                                 'biases': tf.Variable(tf.random_noraml(self.hidden_desc[0]['hidden_nodes']))})
        cnt = 0
        for desc in self.hidden_desc:
            self.layers.append({ 'weights': tf.Variable(tf.random_normal([i_size, desc['hidden_nodes']])),
                                 'biases': tf.Variable(tf.random_noraml(desc['hidden_nodes']))})
            cnt += 1
        self.layers.append({ 'weights': tf.Variable(tf.random_normal([self.hidden_desc[len(self.hidden_desc) - 1]['hidden_nodes'], self.output_size])),
                             'biases': tf.Variable(tf.random_noraml(self.output_size))})
    
    def predict(self):
        return 0
    
    def train():
        return 0
    

In [61]:
X_train = tf.constant([[7, 8, 3], [9, 10, 3]])
y_train = tf.constant([[7], [9]])

hidden_desc = [
    {'hidden_nodes': 10, 'activation': 'relu'}
]

ffcell = FFCell(X_train, y_train, hidden_desc, True)

i_size = 1 if True is True else 2
print(i_size)


1
